In [1]:
import os
%pwd

'/media/sasan/Common/ML/EtoE/research'

In [2]:
os.chdir('../')
%pwd

'/media/sasan/Common/ML/EtoE'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngstionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [4]:
from RedWineQualityPrediction.constanst import *
from RedWineQualityPrediction.utils.common import read_yaml, create_derectories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_derectories([self.config.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngstionConfig:
        config = self.config.data_ingestion
        
        create_derectories([config.root_dir])
        
        return DataIngstionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [7]:
import os
import urllib.request as request
import zipfile
from RedWineQualityPrediction import logger
from RedWineQualityPrediction.utils.common import get_size

In [13]:
class DataIngestion:
    def __init__(self, config: DataIngstionConfig):
        self.config = config
        
    def download_data_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Downloaded file: {file_name} with folowing infor: \n {headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file} and size of the file is: {get_size(Path(self.config.local_data_file))}")
    
    
    def unzip_data_file(self):
        """
        zip_file_path: str
        extract the zip file in to the data directory
        Function return none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingetion = DataIngestion(config=data_ingestion_config)
    data_ingetion.download_data_file()
    data_ingetion.unzip_data_file()
except Exception as e:
    raise e

[2025-02-05 21:38:41,742: INFO: common: Reading the yaml file from ::     config/config.yml]
[2025-02-05 21:38:41,745: INFO: common: Reading the yaml file from ::     params.yml]
[2025-02-05 21:38:41,750: INFO: common: Reading the yaml file from ::     schema.yml]
[2025-02-05 21:38:41,752: INFO: common: Directory created at :: artifacts]
[2025-02-05 21:38:41,753: INFO: common: Directory created at :: artifacts/data_ingestion]
[2025-02-05 21:38:43,227: INFO: 3608074687: Downloaded file: artifacts/data_ingestion/data.zip with folowing infor: 
 Connection: close
Content-Length: 26176
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f751ec0917e15a3dc07c3094a49fb99713109dd8a9f29150bb5d187ec17facdf"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 94BF:197F1B:3BF573:3DD8FB:67A38B67
Accept-Ranges: b